In [1]:
import pandas as pd
import os
import numpy as np
import librosa
from sklearn.preprocessing import OneHotEncoder

In [2]:
paths=[]
labels=[]
for filename in os.listdir('./AudioWAV'):
    
    paths.append('./AudioWAV/' + filename)
    file = filename.split('.')[0]
   
    label = file.split('_')[2]
    if label == 'ANG':
        labels.append('angry.wav')
    elif label == 'DIS':
        labels.append('disgust.wav')
    elif label == 'FEA':
        labels.append('fear.wav')
    elif label == 'HAP':
        labels.append('happy.wav')
    elif label == 'NEU':
        labels.append('neutral.wav')
    elif label == 'SAD':
        labels.append('sad.wav')
        

df_cremad = pd.DataFrame({'speech':paths,'label':labels})
df_cremad.sample(5)

,speech,label
6422,./AudioWAV/1079_IWW_ANG_XX.wav,angry.wav
539,./AudioWAV/1007_IWW_FEA_XX.wav,fear.wav
5635,./AudioWAV/1069_WSI_FEA_XX.wav,fear.wav
728,./AudioWAV/1010_DFA_HAP_XX.wav,happy.wav
1979,./AudioWAV/1025_ITH_FEA_XX.wav,fear.wav


In [3]:
def MFCC(filename):
    y, sr = librosa.load(filename,duration=3,offset=0.5)
    return np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40).T,axis=0)

mfcc_cremad = df_cremad['speech'].apply(lambda x:MFCC(x))

In [4]:
X =[x for x in mfcc_cremad]
X =np.array(X)
X.shape
X =np.expand_dims(X,-1)
X.shape

(7442, 40, 1)

In [5]:
ohe=OneHotEncoder()
y = ohe.fit_transform(df_cremad[['label']] )
y = y.toarray()

In [6]:
X.shape, y.shape

((7442, 40, 1), (7442, 6))

In [7]:
df_cremad['label'].unique()

array(['angry.wav', 'disgust.wav', 'fear.wav', 'happy.wav', 'neutral.wav',
       'sad.wav'], dtype=object)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Definindo os modelos

class SimpleDNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleDNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class LSTMAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        attn_weights = torch.softmax(self.attention(out), dim=1)
        out = torch.sum(attn_weights * out, dim=1)
        out = self.fc(out)
        return out

class CNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.fc_input_size = 32 * 1 * 1
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.dropout(x)
        x = x.view(-1, self.fc_input_size)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class CNNAttention(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNAttention, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.attention = nn.Linear(32, 1)
        self.fc = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        attn_weights = self.softmax(self.attention(x.permute(0, 2, 1))).squeeze(-1)
        attn_weights = attn_weights.unsqueeze(-1)
        x = torch.sum(attn_weights * x, dim=2)
        x = self.fc(x)
        return x
    
# Construindo e treinando os modelos

input_size = X.shape[1:]
num_classes = y.shape[1]
hidden_size = 64
num_layers = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convertendo os dados para tensores PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

# Definindo o tamanho do lote
batch_size = 32

# Criando conjuntos de dados PyTorch
dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Definindo tensor para o LSTM
X_tensorLSTM = X_tensor.permute(0, 2, 1)

# Criando conjuntos de dados PyTorch para LSTM
datasetLSTM = torch.utils.data.TensorDataset(X_tensorLSTM, y_tensor)
train_sizeLSTM = int(0.8 * len(datasetLSTM))
test_sizeLSTM = len(datasetLSTM) - train_sizeLSTM
train_datasetLSTM, test_datasetLSTM = torch.utils.data.random_split(datasetLSTM, [train_sizeLSTM, test_sizeLSTM])

# DataLoader para o LSTM
train_loaderLSTM = DataLoader(train_datasetLSTM, batch_size=batch_size, shuffle=True)
test_loaderLSTM = DataLoader(test_datasetLSTM, batch_size=batch_size, shuffle=False)

# Convertendo os dados para tensores PyTorch do CNN
X_tensorCNN = torch.tensor(X, dtype=torch.float32).to(device)

# Criando conjuntos de dados PyTorch do CNN
datasetCNN = torch.utils.data.TensorDataset(X_tensorCNN, y_tensor)
train_sizeCNN = int(0.8 * len(datasetCNN))
test_sizeCNN = len(datasetCNN) - train_sizeCNN
train_datasetCNN, test_datasetCNN = torch.utils.data.random_split(datasetCNN, [train_sizeCNN, test_sizeCNN])

# DataLoader para o CNN
train_loaderCNN = DataLoader(train_datasetCNN, batch_size=batch_size, shuffle=True)
test_loaderCNN = DataLoader(test_datasetCNN, batch_size=batch_size, shuffle=False)

# Função para treinamento
def train(model, train_loader, criterion, optimizer, num_epochs=500):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            #print(outputs.shape)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

        epoch_loss = running_loss / len(train_loader)
        accuracy = correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')

In [9]:
# Standard Deep Neural Network
sdnn_model = SimpleDNN(input_size[0], num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(sdnn_model.parameters(), lr=0.001)
train(sdnn_model, train_loader, criterion, optimizer)

Epoch [1/500], Loss: 2.9579, Accuracy: 0.1626
Epoch [2/500], Loss: 1.8077, Accuracy: 0.1596
Epoch [3/500], Loss: 1.7907, Accuracy: 0.1957
Epoch [4/500], Loss: 1.7651, Accuracy: 0.2243
Epoch [5/500], Loss: 1.7385, Accuracy: 0.2427
Epoch [6/500], Loss: 1.7248, Accuracy: 0.2573
Epoch [7/500], Loss: 1.6949, Accuracy: 0.2644
Epoch [8/500], Loss: 1.6773, Accuracy: 0.2699
Epoch [9/500], Loss: 1.6504, Accuracy: 0.2792
Epoch [10/500], Loss: 1.6467, Accuracy: 0.2925
Epoch [11/500], Loss: 1.6263, Accuracy: 0.3042
Epoch [12/500], Loss: 1.6099, Accuracy: 0.3111
Epoch [13/500], Loss: 1.6085, Accuracy: 0.3035
Epoch [14/500], Loss: 1.6018, Accuracy: 0.3057
Epoch [15/500], Loss: 1.5889, Accuracy: 0.3130
Epoch [16/500], Loss: 1.5841, Accuracy: 0.3296
Epoch [17/500], Loss: 1.5752, Accuracy: 0.3242
Epoch [18/500], Loss: 1.5699, Accuracy: 0.3329
Epoch [19/500], Loss: 1.5751, Accuracy: 0.3363
Epoch [20/500], Loss: 1.5581, Accuracy: 0.3316
Epoch [21/500], Loss: 1.5562, Accuracy: 0.3465
Epoch [22/500], Loss: 

In [10]:
# LSTM
lstm_model = LSTMModel(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)
train(lstm_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/500], Loss: 1.6525, Accuracy: 0.2956
Epoch [2/500], Loss: 1.5594, Accuracy: 0.3355
Epoch [3/500], Loss: 1.5317, Accuracy: 0.3536
Epoch [4/500], Loss: 1.5332, Accuracy: 0.3660
Epoch [5/500], Loss: 1.4989, Accuracy: 0.3687
Epoch [6/500], Loss: 1.4805, Accuracy: 0.3872
Epoch [7/500], Loss: 1.4692, Accuracy: 0.3971
Epoch [8/500], Loss: 1.4510, Accuracy: 0.4047
Epoch [9/500], Loss: 1.4465, Accuracy: 0.4112
Epoch [10/500], Loss: 1.4298, Accuracy: 0.4136
Epoch [11/500], Loss: 1.4275, Accuracy: 0.4216
Epoch [12/500], Loss: 1.4108, Accuracy: 0.4237
Epoch [13/500], Loss: 1.4091, Accuracy: 0.4243
Epoch [14/500], Loss: 1.4115, Accuracy: 0.4321
Epoch [15/500], Loss: 1.3988, Accuracy: 0.4435
Epoch [16/500], Loss: 1.4013, Accuracy: 0.4208
Epoch [17/500], Loss: 1.3930, Accuracy: 0.4352
Epoch [18/500], Loss: 1.3837, Accuracy: 0.4460
Epoch [19/500], Loss: 1.3848, Accuracy: 0.4381
Epoch [20/500], Loss: 1.3746, Accuracy: 0.4420
Epoch [21/500], Loss: 1.3585, Accuracy: 0.4507
Epoch [22/500], Loss: 

In [11]:
# LSTM with Attention
lstm_atn_model = LSTMAttention(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_atn_model.parameters(), lr=0.001)
train(lstm_atn_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/500], Loss: 1.6638, Accuracy: 0.2861
Epoch [2/500], Loss: 1.5629, Accuracy: 0.3395
Epoch [3/500], Loss: 1.5366, Accuracy: 0.3576
Epoch [4/500], Loss: 1.5258, Accuracy: 0.3593
Epoch [5/500], Loss: 1.4951, Accuracy: 0.3790
Epoch [6/500], Loss: 1.4913, Accuracy: 0.3822
Epoch [7/500], Loss: 1.4556, Accuracy: 0.3976
Epoch [8/500], Loss: 1.4489, Accuracy: 0.3995
Epoch [9/500], Loss: 1.4334, Accuracy: 0.4084
Epoch [10/500], Loss: 1.4430, Accuracy: 0.4104
Epoch [11/500], Loss: 1.4253, Accuracy: 0.4220
Epoch [12/500], Loss: 1.4167, Accuracy: 0.4221
Epoch [13/500], Loss: 1.4112, Accuracy: 0.4215
Epoch [14/500], Loss: 1.4225, Accuracy: 0.4102
Epoch [15/500], Loss: 1.3929, Accuracy: 0.4250
Epoch [16/500], Loss: 1.3937, Accuracy: 0.4297
Epoch [17/500], Loss: 1.4005, Accuracy: 0.4255
Epoch [18/500], Loss: 1.3770, Accuracy: 0.4403
Epoch [19/500], Loss: 1.3792, Accuracy: 0.4369
Epoch [20/500], Loss: 1.3718, Accuracy: 0.4378
Epoch [21/500], Loss: 1.3773, Accuracy: 0.4415
Epoch [22/500], Loss: 

In [12]:
# Convolutional Neural Network
cnn_model = CNNModel(input_size[0],num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)
train(cnn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/500], Loss: 1.7172, Accuracy: 0.2668
Epoch [2/500], Loss: 1.6129, Accuracy: 0.3057
Epoch [3/500], Loss: 1.5917, Accuracy: 0.3156
Epoch [4/500], Loss: 1.5699, Accuracy: 0.3301
Epoch [5/500], Loss: 1.5563, Accuracy: 0.3331
Epoch [6/500], Loss: 1.5521, Accuracy: 0.3328
Epoch [7/500], Loss: 1.5397, Accuracy: 0.3380
Epoch [8/500], Loss: 1.5352, Accuracy: 0.3486
Epoch [9/500], Loss: 1.5365, Accuracy: 0.3442
Epoch [10/500], Loss: 1.5158, Accuracy: 0.3457
Epoch [11/500], Loss: 1.5150, Accuracy: 0.3595
Epoch [12/500], Loss: 1.5219, Accuracy: 0.3548
Epoch [13/500], Loss: 1.5273, Accuracy: 0.3484
Epoch [14/500], Loss: 1.5159, Accuracy: 0.3533
Epoch [15/500], Loss: 1.5046, Accuracy: 0.3669
Epoch [16/500], Loss: 1.5048, Accuracy: 0.3591
Epoch [17/500], Loss: 1.4893, Accuracy: 0.3714
Epoch [18/500], Loss: 1.5006, Accuracy: 0.3687
Epoch [19/500], Loss: 1.4858, Accuracy: 0.3776
Epoch [20/500], Loss: 1.4824, Accuracy: 0.3791
Epoch [21/500], Loss: 1.4703, Accuracy: 0.3796
Epoch [22/500], Loss: 

In [13]:
# Convolutional Neural Network with Attention
cnn_atn_model = CNNAttention(input_size[0],num_classes).to(device)
optimizer = optim.Adam(cnn_atn_model.parameters(), lr=0.001)
train(cnn_atn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/500], Loss: 1.6519, Accuracy: 0.2973
Epoch [2/500], Loss: 1.5633, Accuracy: 0.3492
Epoch [3/500], Loss: 1.5330, Accuracy: 0.3622
Epoch [4/500], Loss: 1.5072, Accuracy: 0.3793
Epoch [5/500], Loss: 1.4887, Accuracy: 0.3828
Epoch [6/500], Loss: 1.4801, Accuracy: 0.3890
Epoch [7/500], Loss: 1.4613, Accuracy: 0.3998
Epoch [8/500], Loss: 1.4469, Accuracy: 0.4033
Epoch [9/500], Loss: 1.4372, Accuracy: 0.4087
Epoch [10/500], Loss: 1.4247, Accuracy: 0.4129
Epoch [11/500], Loss: 1.4175, Accuracy: 0.4223
Epoch [12/500], Loss: 1.4114, Accuracy: 0.4196
Epoch [13/500], Loss: 1.3934, Accuracy: 0.4280
Epoch [14/500], Loss: 1.3965, Accuracy: 0.4356
Epoch [15/500], Loss: 1.3880, Accuracy: 0.4361
Epoch [16/500], Loss: 1.4011, Accuracy: 0.4237
Epoch [17/500], Loss: 1.3936, Accuracy: 0.4322
Epoch [18/500], Loss: 1.3838, Accuracy: 0.4341
Epoch [19/500], Loss: 1.3676, Accuracy: 0.4467
Epoch [20/500], Loss: 1.3517, Accuracy: 0.4440
Epoch [21/500], Loss: 1.3435, Accuracy: 0.4536
Epoch [22/500], Loss: 

In [14]:
# Função para teste
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

    test_loss = running_loss / len(test_loader)
    test_accuracy = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

In [15]:
print("Testing SimpleDNN:")
test(sdnn_model, test_loader, criterion)

Testing SimpleDNN:
Test Loss: 1.4043, Test Accuracy: 0.4332


In [16]:
print("\nTesting LSTMModel:")
test(lstm_model, test_loaderLSTM, criterion)



Testing LSTMModel:
Test Loss: 2.1206, Test Accuracy: 0.4043


In [17]:
print("\nTesting LSTMAttention:")
test(lstm_atn_model, test_loaderLSTM, criterion)


Testing LSTMAttention:


Test Loss: 2.2299, Test Accuracy: 0.4063


In [18]:
print("\nTesting CNNModel:")
test(cnn_model, test_loaderCNN, criterion)



Testing CNNModel:
Test Loss: 1.4930, Test Accuracy: 0.4117


In [19]:
print("\nTesting CNNAttention:")
test(cnn_atn_model, test_loaderCNN, criterion)


Testing CNNAttention:
Test Loss: 2.1666, Test Accuracy: 0.4144
